In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jul 12 11:10:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install pytorch
!pip install torchvision
!pip install opencv-python
!pip install --upgrade Pillow
!pip install scipy
!pip install tqdm
!pip install tensorboardx
!pip install Biopython
# conda install -c menpo opencv : opencv 3.4
# pillow : conda install pillow =6
# conda install scipy
# conda install -c conda-forge tqdm
# conda install -c conda-forge tensorboardx

  Using cached https://files.pythonhosted.org/packages/ee/67/f403d4ae6e9cd74b546ee88cccdb29b8415a9c1b3d80aebeb20c9ea91d96/pytorch-1.0.2.tar.gz
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-nnpl2lcb/pytorch/setup.py'"'"'; __file__='"'"'/tmp/pip-install-nnpl2lcb/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-oume9wzs/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
Requirement already up-to-date: Pillow in /usr/local/lib/python3.6/dist-packages (7.2.0)


In [4]:
import torch
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [5]:
import torch as torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchviz import make_dot
from torch.autograd import Variable
#from torchsummary import summary #make this work
import os
#os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
#from graphviz import Digraph
import torch.nn.utils.prune as prune
import math

In [6]:
class BaseNetCNN(nn.Module):
    """
    A base Network, 81+81+81 parameters in each BaseNet. 
    """
    def __init__(self,channels=32):
        inp=3
        super(BaseNetCNN, self).__init__()
        self.channels=channels
        kernel_size=3
        BatchNorm=nn.BatchNorm2d
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size,padding=1, bias=False)
        self.bn1 = BatchNorm(channels)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size,padding=1, bias=False)
        self.bn2 = BatchNorm(channels)
        self.conv3 = nn.Conv2d(in_channels=channels, out_channels=channels,kernel_size=kernel_size, padding=1, bias=False)
        self.bn3 = BatchNorm(channels)
        self.relu = nn.ReLU()

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu(out)
        
        out += x

        return out

# class BaseNetCNN(nn.Module):

#     def __init__(self, channels):
#         super(BaseNetCNN, self).__init__()
#         self.channels=channels
#         stride=1
#         dilation=1 
#         downsample=None
#         BatchNorm=nn.BatchNorm2d
#         out_channels=channels*2
#         self.conv1 = nn.Conv2d(channels, out_channels, kernel_size=1, bias=False)
#         self.bn1 = BatchNorm(out_channels)
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride,
#                                dilation=dilation, padding=dilation, bias=False)
#         self.bn2 = BatchNorm(out_channels)
#         self.conv3 = nn.Conv2d(out_channels, channels, kernel_size=1, bias=False)
#         self.bn3 = BatchNorm(channels)
#         self.relu = nn.ReLU(inplace=True)
#         self.downsample = downsample
#         self.stride = stride
#         self.dilation = dilation

#     def forward(self, x):
#         residual = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.relu(out)

#         out = self.conv3(out)
#         out = self.bn3(out)

#         if self.downsample is not None:
#             residual = self.downsample(x)

#         out += residual
#         out = self.relu(out)

#         return out
    
class InterconnectionCNN(nn.Module):
    """
    A base Network, hope is to give it parameters less than 100. It might not be possible for ANNs.
    """
    def __init__(self,cnnsize):
        super(InterconnectionCNN, self).__init__()
        #newsize
        #run one layer or two// just doing it for 2x2 module size. DAMN.   
        #one idea here was to divide the input here and make it go through a layer, seems unneccesary
        BatchNorm=nn.BatchNorm2d
        self.conv1 = nn.Conv2d(in_channels=cnnsize, out_channels=512, kernel_size=1,padding=0, bias=False)
        self.bn1 = BatchNorm(512)
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=cnnsize, kernel_size=1,padding=0, bias=False)
        self.bn2 = BatchNorm(cnnsize)
        self.relu = nn.ReLU()

    def forward(self, x):
        #concatenate modules output in each column in Modularity and give out x. 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out

class ModularityCNN(nn.Module):
    #modular_hierarchy=0 :level of hierarchy one is at. 
    def __init__(self,args,depth,width,level,basenet_size):
        super(ModularityCNN, self).__init__()
        """
        args(list) : list containing all modules to be sewed
        depth(int) : The expansion depth
        width(int) : The total width = expansion  width**expo_level + linear_width_multiplier*(level-expo_level)
        level(int) : Levels-1 above basenet, i.e. it contains info about interconnection no.
        //especially needed for weight initlization.   
        """

        self.depth=depth
        self.width=width
        self.level=level
        self.basenet_size= basenet_size#32 #check basenet, needs to be same// I have to find way to get this input here from CSN 
        self.cnn_channels=self.basenet_size*(self.width**self.level) #formula just works till there is no width increase in linear exp.
#         print("CNN Channel Count: ",self.cnn_channels)
#         print("Level",level)
        if len(args) != width*depth:
            assert "Error : Module size not right!"
        if type(args) is list:
            i=0
            j=0
            for module in args:  
                self.add_module("module"+str(i), module)
                i+=1
                #add interconnections based on size of depth and width and level? 
                if(i%self.width==0 and self.width>1 and i<((self.width*self.depth)-1)):
                    self.add_module("interconnection"+str(j),InterconnectionCNN(self.cnn_channels))
                    j+=1
        else:#ordered Dict case
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
      
    def init_weights(self,m):
        """
        To be applied to a same level but when clipping it into the bigger one!
        """
        if type(m) == Modularity:
            #load saved weights of level-1 onto the Modularity!
            #maybe its better to load save weights while forming the lower modularity? i.e -1. 
            #That way I won't have to worry about interconnections at all, its just one apply on all sub-modules
            #god damn I am good. 
            pass
    def forward(self, x): 
        #OOOPS, forgot to do batch norm here...
        #define interconnections name as different. 
        foro=[]
        fori=[]
#         print("Level", self.level, "First X split Sections and x.size():",int(list(x.size())[1]/self.width),x.size())
#         print("Split 2's size",fori[1].size())
        fori=torch.split(x,int(list(x.size())[1]/self.width), dim=1)
        module_iterator= self.children()# this would return interconnections by the end. A bit troublesome. Figured out, 
        #need to test this ofc
        for i in range(self.depth): 
            if(i==0):
                for j in range(self.width): 
                    #first layer in given width
                    depthbuff=0
                    if(j==0):
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    elif(j==self.width-1): 
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    else:   
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                
            elif(i==self.depth-1): 
                module=next(module_iterator)# this is interconnection hopefully
                #print(module)
                #print("LEVEL:",self.level,"Above module's concatenated input size: ", torch.cat(foro,1).size())
                inp=torch.cat(foro,1)
                output=module(inp)
                del inp
                del fori
                fori=torch.split(output,int(list(output.size())[1]/self.width),1)
                del output
                depthbuff=0
                del foro
                foro=[]
                for j in range(self.width): 
                    if(j==0):
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    elif(j==self.width-1): 
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    else:   
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
            else:
                module=next(module_iterator)# this is interconnection hopefully
                #check if this module is interconnection type?
#                 print(module)
#                 print("LEVEL:",self.level,"Above module's concatenated input size: ", torch.cat(foro,1).size())
                inp=torch.cat(foro,1)
                output=module(inp)
                del inp
                del fori
                fori=torch.split(output,int(list(output.size())[1]/self.width),1)
                del output
                depthbuff=0
                del foro
                foro=[]
                for j in range(self.width): 
                    
                    if(j==0):
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    elif(j==self.width-1): 
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1
                    else:   
                        module=next(module_iterator)
                        out1=module(fori[depthbuff])
                        foro.append(out1)
                        depthbuff+=1

        out= torch.cat(foro,1)
        del foro
        out += x
        return out

In [7]:
class CompositionalSparseNetCNN(nn.Module):
    def __init__(self,inp=3,stage="Base_model",exp_level=4,channels_width=256, exponential_cutoff=4,\
                 dir_saved_weights=r'Weights/',expo_depth=2,expo_width=2,nettype="CNN",prev_path="CSN"):
        """
        inp[int]: baseNet size
        pruning_type (string) : 'unstructured' or 'structured', structured refers to layers or modules
        stage (string) : 'base_model' or exp_expansion' or 'linear_expansion' or 'reload'
        exp_level (int) : 0-100 : represents total expansion  level including both exponen. and linear.
        exponential_cutoff(int) : 1 to 20, tells expansion level at which exponential expansion halts
        dir_saved_weights (string): filepath to save prev. modular hierarchy level weights for uploading
        expo_depth (int) : exponential expansion depth size
        expo_width (int) : exponential expansion width size
        channels_width(int) : Based on exponential expansion size, this width will change!
        nettype (string) : "CNN" or "FCN" 
        """
#         List of changes to make : 
#             1. Give width of the whole network in MODULARITY. 
#             2. Manage the input filter size with width of current module. 
#            3. Check the fucking variable width and depth. 

        super(CompositionalSparseNetCNN, self).__init__()
        self.inp=inp
        self.exp_level=exp_level
        self.stage=stage
        self.dir_saved_weights=dir_saved_weights
        self.exponential_cutoff=exponential_cutoff
        self.prev_path=prev_path
        self.nettype=nettype
        
        
        if(self.stage=='exp_expansion' and (self.exp_level!=self.exponential_cutoff)):
            assert "Inconsistent Model configuration, At exponentially expansion stage, expansion level should be the same number as exponential exp level"
    
        #merge both exponential and linear expansion methods after weight manipulation.
        #exponential expansion
        self.expo_depth=expo_depth # y=(ed)^level y= x^a
        self.expo_width=expo_width
        #linear expansion
        self.linear_depth=4 # y= multiplier* ld #y=mx 
        self.linear_width=0 #lets keep this constant for a while. BECOZ OF FREAKIN GPUs.  
        self.multiplier=exp_level-self.exponential_cutoff
        self.basenet_size= int(channels_width/(self.expo_width**self.exponential_cutoff + self.linear_width*self.multiplier))
        print(self.basenet_size)
        #first two layers
        self.relu = nn.ReLU()
        self.channels_width=channels_width
        self.downsamp1= nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=7, stride=2, padding=3,
                                bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            )
        self.conv1= nn.Sequential(
            nn.Conv2d(256, channels_width, kernel_size=1, stride=1, padding=0,
                                bias=False),
            nn.BatchNorm2d(channels_width),
            nn.ReLU(inplace=True)
            )

        #transfer contents to a method. 
        total_width = self.expo_width**exponential_cutoff + (exp_level-exponential_cutoff)*self.linear_width
        all_modules=self.expansion()
        #modularityCNN definition
        if(self.multiplier>0): 
            #perform addition of modules similar to above!. 
            td= self.expo_depth+self.linear_depth*self.multiplier
            tw=self.expo_width+self.linear_width*self.multiplier
            self.modularity=ModularityCNN(all_modules,td,tw,self.exponential_cutoff,self.basenet_size)
        else :
            self.modularity=ModularityCNN(all_modules,self.expo_depth,self.expo_width,self.exponential_cutoff,self.basenet_size)
        if(self.stage=='linear_expansion'): 
            #loading dicts from previous expansion levels. 
            #PUT IN A CHECK IF LEVEL 1 IS NOT THERE, JUST PUT EXPODICT AS rand weight init
            path=self.prev_path+'/'+'level'+str(self.exponential_cutoff-1)+'.pth'
            statedict=torch.load(path)     
            expodict={}
            for k in list(statedict.keys()):
                lst=k.split('.')
                if(lst[0]=="modularity"):
                    lst=lst[1:]
                    new_k='.'.join(lst)
                    expodict[new_k]=statedict[k]
            
            old_depth=self.expo_depth + self.linear_depth*(self.multiplier-1)
            old_width=self.expo_width + self.linear_width*(self.multiplier-1)
            new_depth=self.expo_depth + self.linear_depth*self.multiplier
            new_width=self.expo_width + self.linear_width*self.multiplier
            
            path=self.prev_path+'/'+'level'+str(self.exp_level-1)+'.pth'
            statedict=torch.load(path)     
            lineardict= [{}] * (old_depth*old_width)
            linearinter=[{}] * (old_depth-1)
            for k in list(statedict.keys()):
                lst=k.split('.')
                if(lst[0]=="modularity"):
                    lst=lst[1:]
                    no=0
                    if lst[0][0]=="m": 
                        no=int(lst[0][6:])#getting no. from module..no
                        new_k='.'.join(lst[1:]) #removes modularity right? but it contains module0 identity of itself?
                        lineardict[no][new_k]=statedict[k] 
                    elif lst[0][0]=="i": 
                        no=int(lst[0][15:])#getting no. from interconnection..no
                        new_k='.'.join(lst[1:]) #removes modularity right? but it contains module0 identity of itself?
                        linearinter[no][new_k]=statedict[k]
                    else : 
                        assert "The loaded linear dict is not right!"
                    
        
            #print('lineardict', [dit.keys() for dit in lineardict])
            #print('linearinter',linearinter)

            width_pos=math.ceil(float(new_width - old_width)/2)
            depth_pos=math.ceil(float(new_depth - old_depth)/2)
            first_num=depth_pos*new_width + width_pos
            total_lst=[i for i in range(0,new_depth*new_width)]
            lst_linear=[]
            lst_exp=[]
            #can find interconnection no. based on depth_pos simply. Its depth_pos-1? 
            
            for j in range(0,old_depth):
                lst_linear.extend([i+first_num+j*new_width for i in range(0,old_width)])
            print(lst_linear)
            lst_exp=[item for item in total_lst if item not in lst_linear]
            print(lst_exp)
            lin_buffer=0
            for n,module in enumerate(self.modularity.named_children()):
                name=module[0]
                mtype= "x"
                no=0
                if name[0]=="m": 
                    no=int(name[6:])#getting no. from module..no
                    mtype="m"
                elif name[0]=="i": 
                    no=int(name[15:])#getting no. from interconnection..no
                    mtype="i"
                else : 
                    no=None
                    print(no,name)
                    print(mtype)
                    continue
                print(no,name)
                print(mtype)  
                
                if(mtype=="m"):
                    if (no in lst_linear):
                        module[1].load_state_dict(lineardict[lin_buffer])
                        lin_buffer+=1
                        print("Checking linBuffer and module no./decrease intercon count",lin_buffer,n)
                    elif no in lst_exp: 
                        module[1].load_state_dict(expodict)
                elif(mtype=="i") and (old_width==new_width):
                    #there is no way interconnections are gonna work through transfer learning if width is increased.
                    inter_num=int(((lin_buffer)/old_width)-1)
                    if(0 < lin_buffer < max(lst_linear)) and inter_num < len(linearinter):
                        print("Checking internum and module no./decrease intercon count",inter_num,n)
                        module[1].load_state_dict(linearinter[inter_num]) 

                else : 
                    assert "What kind of module is this? it should not be here."
        """BIGGEST MISTAKE Here could be that, the loading of state dict does not happen?"""
        #print("self.modularity.state_dict: ",self.modularity.state_dict().keys())

        self.downsamp2 = nn.Sequential(
            nn.Conv2d(channels_width,1024, kernel_size=7, stride=2, padding=3,
                                bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.Conv2d(1024, 2048, kernel_size=7, stride=2, padding=3,
                                bias=False),
            nn.BatchNorm2d(2048),
            nn.ReLU(inplace=True),
            )
    
    def expansion(self): 
        """
        creates a basenet list and stiches them into Modularity Classes design. 
        """
        
        exp_basenet_list = [BaseNetCNN(self.basenet_size) for count in range(0,(self.expo_depth*self.expo_width)**self.exponential_cutoff)]
        #was not thinking, need to load weights on those Modularity modules which are one_level below! see if thats possible.
        module_size=self.expo_depth*self.expo_width
        for i in range(0,self.exponential_cutoff-1):
            new_basenet=[]
            for j in range(0,(module_size)**(self.exponential_cutoff-i),module_size):
                new_basenet.append(ModularityCNN(exp_basenet_list[j:j+module_size],self.expo_depth,self.expo_width,i+1,self.basenet_size))
            exp_basenet_list=new_basenet

        if(self.stage =='exp_expansion'): 
            for i in range(len(exp_basenet_list)):
                path=self.prev_path+'/'+'level'+str(self.exponential_cutoff-1)+'.pth'
                statedict=torch.load(path)              
                #In order to load this
                """I just realized I have to write this script in weight init method or it will be just overwritten right
                ??"""
                modulardict={}
                for k in list(statedict.keys()):
                    lst=k.split('.')
                    if(lst[0]=="modularity"):
                        lst=lst[1:]
                        new_k='.'.join(lst)
                        modulardict[new_k]=statedict[k]
                exp_basenet_list[i].load_state_dict(modulardict)
                #print(exp_basenet_list[i].state_dict())

        lin_basenet_list=[]
        #linear expansion
        if(self.multiplier>0): 
            #td = total depth of last level of dynamic hierarchy 
            #tw =  total width of last level of dynamic hierarchy
            td= self.expo_depth+self.linear_depth*self.multiplier
            tw=self.expo_width+self.linear_width*self.multiplier
            d=self.expo_depth
            w=self.expo_width
            lin_basenet_list = [BaseNetCNN(self.basenet_size) for count in range(0,((d*w)**(self.exponential_cutoff-1))*(td*tw-d*w))]
        
            module_size=d*w
            final_module_size=td*tw
            for i in range(0,self.exponential_cutoff-1):
                new_basenet=[]
                j=0
                while(j<len(lin_basenet_list)):
                    new_basenet.append(ModularityCNN(lin_basenet_list[j:j+module_size],self.expo_depth,self.expo_width,i+1,self.basenet_size))
                    j+=module_size                        
                lin_basenet_list=new_basenet

        basenet_list= exp_basenet_list+lin_basenet_list
        return basenet_list
    
    def forward(self, x):
        """based on hierarchy levels, depths for linear and exponential expansion, 
        base net is repeated in init with various self.basenet in list and then used to be trained."""
        x=self.downsamp1(x)
        low_level_feat=x
        x=self.conv1(x)
        x=self.modularity(x)
        x=self.downsamp2(x)
        return x, low_level_feat
    

In [8]:
def global_pruning(model,pruning_rate=0.3):
        """
        Global unstructured magnitude L1 norm pruning
        design or experiment choices : to create pruning on top of pruning or remove old ones to produce new higher rates? 
        Current : Keeps pruning by keeping weights zero! but next pruning will just prune these zeroes too right, need to increase
        the rate still, so the other choice is actually better. To just restore everything back to normal. shall do that then
        """
        parameters_to_prune={}
        for name, param in model.named_parameters():
            #parameters_to_prune[name]= param.data
            lst=name.split('.')            
            s="."
            LOC=s.join(["model"]+lst[0:-1])
            if(lst[0]=='modularity'):
                exec("parameters_to_prune["+LOC+"]= 'weight'")#just pruning weights, not biases/thats how its done.
                
        parameter_tuple=tuple(list(parameters_to_prune.items()))
        prune.global_unstructured(
             parameter_tuple,
             pruning_method=prune.L1Unstructured,
             amount=pruning_rate,)

def remove_global_pruning(model): 
        parameters_to_prune={}
        for name, param in model.named_parameters():
            #parameters_to_prune[name]= param.data
            lst=name.split('.')            
            s="."
            LOC=s.join(["model"]+lst[0:-1])
            if(lst[0]=='modularity'):
                exec("parameters_to_prune["+LOC+"]= 'weight'")#just pruning weights, not biases/thats how its done.
        
        parameter_tuple=tuple(list(parameters_to_prune.items())) 
        for m in parameter_tuple: 
            #does the job really, removed all pruning reparametrization but keeps zeroes in weights. 
            #print(m)
            prune.remove(m[0], 'weight')
            
def structured_pruning(model, pruning_rate=0.3): 
    """Acts on all BaseNets differently!"""
    basenets_to_prune=[]
    for name, param in model.named_parameters():
        #parameters_to_prune[name]= param.data
        lst=name.split('.')            
        s="."
        LOC=s.join(["model"]+lst[0:-1])
        #print(lst[-2])
        if(lst[0]=='modularity') and lst[-2][0:4]=='conv':
            basenets_to_prune.append(LOC)
            #print(LOC)
    basenets_to_prune=list(set(basenets_to_prune))
    for basenets in basenets_to_prune:
        exec("prune.ln_structured("+basenets+", name='weight', amount="+str(pruning_rate)+", n=2, dim=0)")

def remove_structured_pruning(model): 
    """Acts on all BaseNets differently!"""
    basenets_to_prune=[]
    for name, param in model.named_parameters():
        #parameters_to_prune[name]= param.data
        lst=name.split('.')            
        s="."
        LOC=s.join(["model"]+lst[0:-1])
        #print(lst[-2])
        if(lst[0]=='modularity') and lst[-2][0:4]=='conv':
            basenets_to_prune.append(LOC)
            print(LOC)
    basenets_to_prune=list(set(basenets_to_prune))
    for basenets in basenets_to_prune: 
        exec("prune.remove("+basenets+",'weight')")


def randomize_pruning(model,ranmodel):
    """WORKS do it only for modularity though. Although 0.8*random1 + 0.2*random2 is still random..!"""
    beta = 0.2 #The interpolation parameter    
    dict1=model.modularity.state_dict()
    dict2=ranmodel.modularity.state_dict()
    dict3={}
    #print(dict1)
    #print(dict1.keys())
    for key in dict1.keys(): 
        dict3[key]=beta*(dict2[key])+(1-beta)*dict1[key]
    #print(dict3)
    model.modularity.load_state_dict(dict3)
    return model
            
    #parameter_tuple=tuple(list(parameters_to_prune.items()))
    #print(parameter_tuple)
    #for item in parameter_tuple: 
        
def create_graph(model,inp):
    nput = torch.randn(1, *inp)
    #transforms = [
        # Fold Conv, BN, RELU layers into one
        #hl.transforms.Fold("Transpose > MatMul > Add > Relu", "FCLayer",name="FCLayer"),
        # Fold repeated blocks
       #  hl.transforms.FoldDuplicates(),]
#     hl.build_graph(model, nput,transforms=transforms).save("newone",format="pdf")
    hl.build_graph(model, nput).save("newone",format="pdf")

In [9]:
%cd gdrive/My\ Drive/CSNModels/DeepLab/
!ls

/content/gdrive/My Drive/CSNModels/DeepLab
Data	     DeepLabCSN  mypath.py    run	utils
dataloaders  modeling	 __pycache__  train.py


In [10]:
import argparse
import os
import numpy as np
from tqdm import tqdm

from dataloaders import make_data_loader
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *
from utils.loss import SegmentationLosses
from utils.calculate_weights import calculate_weigths_labels
from utils.lr_scheduler import LR_Scheduler
from utils.saver import Saver
from utils.summaries import TensorboardSummary
from utils.metrics import Evaluator
import sys
class Trainer(object):
    def __init__(self, args):#,model):
        self.args = args

        # Define Saver
        self.saver = Saver(args)
        self.saver.save_experiment_config()
        # Define Tensorboard Summary
        self.summary = TensorboardSummary(self.saver.experiment_dir)
        self.writer = self.summary.create_summary()
        
        # Define Dataloader
        kwargs = {'num_workers': args.workers, 'pin_memory': True}
        self.train_loader, self.val_loader, self.test_loader, self.nclass = make_data_loader(args, **kwargs)
        #backbone = model
        # Define network
        model = DeepLab(num_classes=self.nclass,
                        backbone=args.backbone,
                        output_stride=args.out_stride,
                        sync_bn=args.sync_bn,
                        freeze_bn=args.freeze_bn)

        train_params = [{'params': model.get_1x_lr_params(), 'lr': args.lr},
                        {'params': model.get_10x_lr_params(), 'lr': args.lr * 10}]

        # Define Optimizer
        optimizer = torch.optim.SGD(train_params, momentum=args.momentum,
                                    weight_decay=args.weight_decay, nesterov=args.nesterov)

        # Define Criterion
        # whether to use class balanced weights
        if args.use_balanced_weights:
            classes_weights_path = os.path.join(Path.db_root_dir(args.dataset), args.dataset+'_classes_weights.npy')
            if os.path.isfile(classes_weights_path):
                weight = np.load(classes_weights_path)
            else:
                weight = calculate_weigths_labels(args.dataset, self.train_loader, self.nclass)
            weight = torch.from_numpy(weight.astype(np.float32))
        else:
            weight = None
        self.criterion = SegmentationLosses(weight=weight, cuda=args.cuda).build_loss(mode=args.loss_type)
        self.model, self.optimizer = model, optimizer
        
        # Define Evaluator
        self.evaluator = Evaluator(self.nclass)
        # Define lr scheduler
        self.scheduler = LR_Scheduler(args.lr_scheduler, args.lr,
                                            args.epochs, len(self.train_loader))

        # Using cuda
        if args.cuda:
            self.model = torch.nn.DataParallel(self.model, device_ids=self.args.gpu_ids)
            patch_replication_callback(self.model)
            self.model = self.model.cuda()

        # Resuming checkpoint
        self.best_pred = 0.0
        if args.resume is not None:
            if not os.path.isfile(args.resume):
                raise RuntimeError("=> no checkpoint found at '{}'" .format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            if args.cuda:
                self.model.module.load_state_dict(checkpoint['state_dict'])
            else:
                self.model.load_state_dict(checkpoint['state_dict'])
            if not args.ft:
                self.optimizer.load_state_dict(checkpoint['optimizer'])
            self.best_pred = checkpoint['best_pred']
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))

        # Clear start epoch if fine-tuning
        if args.ft:
            args.start_epoch = 0

    def training(self, epoch):
        train_loss = 0.0
        self.model.train()
        tbar = tqdm(self.train_loader)
        num_img_tr = len(self.train_loader)
        for i, sample in enumerate(tbar):
            image, target = sample['image'], sample['label']
            if self.args.cuda:
                image, target = image.cuda(), target.cuda()
            self.scheduler(self.optimizer, i, epoch, self.best_pred)
            self.optimizer.zero_grad()
            output = self.model(image)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
            tbar.set_description('Train loss: %.3f' % (train_loss / (i + 1)))
            self.writer.add_scalar('train/total_loss_iter', loss.item(), i + num_img_tr * epoch)

            # Show 10 * 3 inference results each epoch
            if i % (num_img_tr // 10) == 0:
                global_step = i + num_img_tr * epoch
                #print("I was here!!")
                self.summary.visualize_image(self.writer, self.args.dataset, image, target, output, global_step)

        self.writer.add_scalar('train/total_loss_epoch', train_loss, epoch)
        print('[Epoch: %d, numImages: %5d]' % (epoch, i * self.args.batch_size + image.data.shape[0]))
        print('Loss: %.3f' % train_loss)

        if self.args.no_val:
            # save checkpoint every epoch
            is_best = False
            self.saver.save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': self.model.module.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'best_pred': self.best_pred,
            }, is_best)


    def validation(self,epoch,loader='val'):
        self.model.eval()
        self.evaluator.reset()
        if(loader=="val"):
            tbar = tqdm(self.val_loader, desc='\r')
        elif(loader=="test"):
            print('TEST RESULTS ARE BELOW::::::::::::::::::::::::::::')
            tbar = tqdm(self.test_loader, desc='\r')
        test_loss = 0.0
        for i, sample in enumerate(tbar):
            image, target = sample['image'], sample['label']
            if self.args.cuda:
                image, target = image.cuda(), target.cuda()
            with torch.no_grad():
                output = self.model(image)
            loss = self.criterion(output, target)
            test_loss += loss.item()
            tbar.set_description('Test loss: %.3f' % (test_loss / (i + 1)))
            pred = output.data.cpu().numpy()
            target = target.cpu().numpy()
            pred = np.argmax(pred, axis=1)
            # Add batch sample into evaluator
            self.evaluator.add_batch(target, pred)

        # Fast test during the training
        Acc = self.evaluator.Pixel_Accuracy()
        Acc_class = self.evaluator.Pixel_Accuracy_Class()
        mIoU,IoU = self.evaluator.Mean_Intersection_over_Union()
        FWIoU = self.evaluator.Frequency_Weighted_Intersection_over_Union()
        self.writer.add_scalar('val/total_loss_epoch', test_loss, epoch)
        self.writer.add_scalar('val/mIoU', mIoU, epoch)
        self.writer.add_scalar('val/Acc', Acc, epoch)
        self.writer.add_scalar('val/Acc_class', Acc_class, epoch)
        self.writer.add_scalar('val/fwIoU', FWIoU, epoch)
        print('Validation:')
        print('[Epoch: %d, numImages: %5d]' % (epoch, i * self.args.batch_size + image.data.shape[0]))
        print("Acc:{}, Acc_class:{}, mIoU:{}, fwIoU: {}".format(Acc, Acc_class, mIoU, FWIoU))
        print("Classwise_IoU:")
        print(IoU)
        print('Loss: %.3f' % test_loss)
        print(self.evaluator.confusion_matrix)
        if(loader=='val'):
            new_pred = mIoU
            if new_pred > self.best_pred:
                is_best = True
                self.best_pred = new_pred
                self.saver.save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': self.model.module.state_dict(),
                    'optimizer': self.optimizer.state_dict(),
                    'best_pred': self.best_pred,
                }, is_best)


In [11]:
def main(model):    
    parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")
    parser.add_argument('--backbone', type=str, default='resnet',
                        choices=['resnet', 'xception', 'drn', 'mobilenet', 'csn'],
                        help='backbone name (default: resnet)')
    parser.add_argument('--out-stride', type=int, default=16,
                        help='network output stride (default: 8)')
    parser.add_argument('--dataset', type=str, default='marsh',
                        choices=['pascal', 'coco', 'cityscapes','marsh'],
                        help='dataset name (default: pascal)')
    parser.add_argument('--use-sbd', action='store_true', default=True,
                        help='whether to use SBD dataset (default: True)')
    parser.add_argument('--workers', type=int, default=4,
                        metavar='N', help='dataloader threads')
    parser.add_argument('--base-size', type=int, default=513,
                        help='base image size')
    parser.add_argument('--crop-size', type=int, default=513,
                        help='crop image size')
    parser.add_argument('--sync-bn', type=bool, default=None,
                        help='whether to use sync bn (default: auto)')
    parser.add_argument('--freeze-bn', type=bool, default=False,
                        help='whether to freeze bn parameters (default: False)')
    parser.add_argument('--loss-type', type=str, default='ce',
                        choices=['ce', 'focal'],
                        help='loss func type (default: ce)')
    # training hyper params
    parser.add_argument('--epochs', type=int, default=None, metavar='N',
                        help='number of epochs to train (default: auto)')
    parser.add_argument('--start_epoch', type=int, default=0,
                        metavar='N', help='start epochs (default:0)')
    parser.add_argument('--batch-size', type=int, default=2,
                        metavar='N', help='input batch size for \
                                training (default: auto)')
    parser.add_argument('--test-batch-size', type=int, default=None,
                        metavar='N', help='input batch size for \
                                testing (default: auto)')
    parser.add_argument('--use-balanced-weights', action='store_true', default=False,
                        help='whether to use balanced weights (default: False)')
    # optimizer params
    parser.add_argument('--lr', type=float, default=None, metavar='LR',
                        help='learning rate (default: auto)')
    parser.add_argument('--lr-scheduler', type=str, default='poly',
                        choices=['poly', 'step', 'cos'],
                        help='lr scheduler mode: (default: poly)')
    parser.add_argument('--momentum', type=float, default=0.9,
                        metavar='M', help='momentum (default: 0.9)')
    parser.add_argument('--weight-decay', type=float, default=5e-4,
                        metavar='M', help='w-decay (default: 5e-4)')
    parser.add_argument('--nesterov', action='store_true', default=False,
                        help='whether use nesterov (default: False)')
    # cuda, seed and logging
    parser.add_argument('--no-cuda', action='store_true', default=
                        False, help='disables CUDA training')
    parser.add_argument('--gpu-ids', type=str, default='0',
                        help='use which gpu to train, must be a \
                        comma-separated list of integers only (default=0)')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    # checking point
    parser.add_argument('--resume', type=str, default=None,
                        help='put the path to resuming file if needed')
    parser.add_argument('--checkname', type=str, default=None,
                        help='set the checkpoint name')
    # finetuning pre-trained models
    parser.add_argument('--ft', action='store_true', default=False,
                        help='finetuning on a different dataset')
    # evaluation option
    parser.add_argument('--eval-interval', type=int, default=1,
                        help='evaluuation interval (default: 1)')
    parser.add_argument('--no-val', action='store_true', default=False,
                        help='skip validation during training')

    args = parser.parse_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()
    if args.cuda:
        try:
            args.gpu_ids = [int(s) for s in args.gpu_ids.split(',')]
        except ValueError:
            raise ValueError('Argument --gpu_ids must be a comma-separated list of integers only')

    if args.sync_bn is None:
        if args.cuda and len(args.gpu_ids) > 1:
            args.sync_bn = True
        else:
            args.sync_bn = False

    # default settings for epochs, batch_size and lr
    if args.epochs is None:
        epoches = {
            'coco': 30,
            'cityscapes': 200,
            'pascal': 50,
			'marsh' : 100,
        }
        args.epochs = epoches[args.dataset.lower()]

    if args.batch_size is None:
        args.batch_size = 4 * len(args.gpu_ids)

    if args.test_batch_size is None:
        args.test_batch_size = args.batch_size

    if args.lr is None:
        lrs = {
            'coco': 0.1,
            'cityscapes': 0.01,
            'pascal': 0.007,
			'marsh' : 0.01,
        }
        args.lr = lrs[args.dataset.lower()] / (4 * len(args.gpu_ids)) * args.batch_size


    if args.checkname is None:
        args.checkname = 'deeplab-'+str(args.backbone)
    args.backbone=model
    print(args)
    print('This will also print testing data results.. hopefully...')
    torch.manual_seed(args.seed)
    trainer = Trainer(args)
    print('Starting Epoch:', trainer.args.start_epoch)
    print('Total Epoches:', trainer.args.epochs)
    for epoch in range(trainer.args.start_epoch, trainer.args.epochs):
        trainer.training(epoch)
        if not trainer.args.no_val and epoch % args.eval_interval == (args.eval_interval - 1):
            trainer.validation(epoch)
    trainer.validation(0,loader='test')
    
    trainer.writer.close()

In [12]:
if __name__ == "__main__":
    from Bio import SeqIO
    path = '/path/to/sequences.txt'
    sequences = ['']
    sys.argv = sequences
    stages=['base_model', 'exp_expansion', 'linear_expansion', 'reload']
    path=F"/content/gdrive/My Drive/CSNModels/DeepLab/DeepLabCSN"
    torch.cuda.empty_cache()
    #first was without pruning. This one is without randomized pruning/ but with structured pruning. Lets see/ We have test results of this here too. 
    
    # model=CompositionalSparseNetCNN(exp_level=1, exponential_cutoff=1,expo_depth=2,expo_width=2,stage=stages[0],
    #                         prev_path=path, channels_width=128)
    
    # main(model)
    # structured_pruning(model, pruning_rate=0.5)
    # remove_structured_pruning(model)
    # torch.save(model.state_dict(),path+"/level1.pth")

    # del model
    # model=CompositionalSparseNetCNN(exp_level=2, exponential_cutoff=2,expo_depth=2,expo_width=2,stage=stages[0],
    #                         prev_path=path, channels_width=256)
    
    # main(model)
    # structured_pruning(model, pruning_rate=0.5)
    # remove_structured_pruning(model)
    # torch.save(model.state_dict(),path+"/level2.pth")
    # del model
    torch.cuda.empty_cache()
    model=CompositionalSparseNetCNN(exp_level=3, exponential_cutoff=2,expo_depth=2,expo_width=2,stage=stages[0],
                            prev_path=path, channels_width=256)
    main(model)
    structured_pruning(model, pruning_rate=0.5)
    remove_structured_pruning(model)
    torch.save(model.state_dict(),path+"/level3base.pth")
    # del model
    # torch.cuda.empty_cache()
    # model=CompositionalSparseNetCNN(exp_level=4, exponential_cutoff=2,expo_depth=2,expo_width=2,stage=stages[2],
    #                         prev_path=path, channels_width=256)
    # main(model)
    # structured_pruning(model, pruning_rate=0.5)
    # remove_structured_pruning(model)
    # torch.save(model.state_dict(),path+"/level4.pth")

64
Namespace(backbone=CompositionalSparseNetCNN(
  (relu): ReLU()
  (downsamp1): Sequential(
    (0): Conv2d(3, 256, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv1): Sequential(
    (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (modularity): ModularityCNN(
    (module0): ModularityCNN(
      (module0): BaseNetCNN(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchN

  0%|          | 0/181 [00:00<?, ?it/s]

Starting Epoch: 0
Total Epoches: 100

=>Epoches 0, learning rate = 0.0050,                 previous best = 0.0000


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
:   0%|          | 0/26 [00:00<?, ?it/s]

[Epoch: 0, numImages:   362]
Loss: 91.584


...

[Epoch: 99, numImages:   362]
Loss: 43.157


:   0%|          | 0/56 [00:00<?, ?it/s]

Validation:
[Epoch: 99, numImages:    52]
Acc:0.8033162077483407, Acc_class:0.3023268763849111, mIoU:0.24129149267459743, fwIoU: 0.6698953063052733
Classwise_IoU:
[0.81880124 0.         0.47440178 0.41074791 0.0788518  0.02468283
 0.         0.36319052 0.00094734]
Loss: 8.058
[[9.23099e+06 0.00000e+00 5.58310e+04 4.41220e+04 1.35220e+04 1.54400e+03
  0.00000e+00 1.15165e+05 0.00000e+00]
 [7.21100e+03 0.00000e+00 2.07140e+04 1.09680e+04 0.00000e+00 0.00000e+00
  0.00000e+00 4.96500e+03 0.00000e+00]
 [5.30692e+05 0.00000e+00 8.78872e+05 5.39000e+04 8.08000e+02 2.13600e+03
  0.00000e+00 2.10088e+05 0.00000e+00]
 [2.14858e+05 0.00000e+00 4.92000e+04 4.07220e+05 0.00000e+00 3.23000e+02
  0.00000e+00 6.04880e+04 0.00000e+00]
 [9.00530e+04 0.00000e+00 9.90100e+03 1.06600e+03 1.17680e+04 4.25500e+03
  0.00000e+00 8.69000e+03 0.00000e+00]
 [5.98345e+05 0.00000e+00 2.54200e+04 1.04300e+03 8.71800e+03 1.63350e+04
  0.00000e+00 3.67700e+03 0.00000e+00]
 [1.27130e+04 0.00000e+00 3.72000e+02 3.26800

Test loss: 0.307: 100%|██████████| 56/56 [00:15<00:00,  3.57it/s]


Validation:
[Epoch: 0, numImages:   111]
Acc:0.8104841615323473, Acc_class:0.3176707533434975, mIoU:0.23799330645771746, fwIoU: 0.6885255189983248
Classwise_IoU:
[8.31859612e-01 0.00000000e+00 4.47256266e-01 4.12472358e-01
 1.26540280e-01 1.32262478e-02 0.00000000e+00 3.10578571e-01
 6.42385907e-06]
Loss: 17.203
[[2.0463244e+07 0.0000000e+00 5.0892200e+05 1.5315900e+05 2.6949000e+04
  2.4937000e+04 0.0000000e+00 2.0122600e+05 0.0000000e+00]
 [6.5300000e+03 0.0000000e+00 1.4778000e+04 4.6180000e+03 0.0000000e+00
  0.0000000e+00 0.0000000e+00 9.9750000e+03 0.0000000e+00]
 [9.0643000e+05 0.0000000e+00 1.9072340e+06 1.3142600e+05 5.2440000e+03
  7.3910000e+03 0.0000000e+00 4.1569600e+05 0.0000000e+00]
 [2.2552500e+05 0.0000000e+00 8.8651000e+04 6.9499200e+05 2.6000000e+01
  0.0000000e+00 0.0000000e+00 9.2265000e+04 0.0000000e+00]
 [2.0233800e+05 0.0000000e+00 1.7682000e+04 1.7400000e+02 3.8787000e+04
  1.9500000e+03 0.0000000e+00 8.5670000e+03 0.0000000e+00]
 [1.3345830e+06 0.0000000e+00 8

In [13]:
# TO Do : Do pruning whatever I have right now. 
#better create testing set here too. 